<a href="https://colab.research.google.com/github/oserikov/data-science-nlp/blob/master/BERT%20%D0%B4%D0%BB%D1%8F%20seq2seq%20%7C%20ATIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install seqeval
!pip install pytorch-pretrained-bert

import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=48055439a62b3a0b6efb9d7b68dbe283edbe338c57c73ccf4a727d4b35d935eb
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
     |████████████████████████████████| 133kB 3.4MB/s 
     |████████████████████████████████| 645kB 52.2MB/s 


In [0]:
import pickle
import pandas as pd
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# https://www.kaggle.com/irustandi/bertlargeuncasedwwm-pytorch
!cp /content/drive/My\ Drive/bertlargeuncasedwwm-pytorch.zip .
!unzip -q bertlargeuncasedwwm-pytorch.zip

In [8]:
!wget https://gist.githubusercontent.com/oserikov/d82ab401d278c454e9ff6e8779688f25/raw/268b80699c2daa609252703969bff16f4278d0d5/run_ner_atis.py

--2019-11-07 12:05:51--  https://gist.githubusercontent.com/oserikov/d82ab401d278c454e9ff6e8779688f25/raw/268b80699c2daa609252703969bff16f4278d0d5/run_ner_atis.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25374 (25K) [text/plain]
Saving to: ‘run_ner_atis.py’

run_ner_atis.py     100%[===================>]  24.78K  --.-KB/s    in 0.01s   

2019-11-07 12:05:51 (2.38 MB/s) - ‘run_ner_atis.py’ saved [25374/25374]



In [9]:
%cp -r drive/My\ Drive/atis .
%cd atis

/content/atis


### предобработка датасета ATIS

In [10]:
!nvidia-smi

Thu Nov  7 12:08:56 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [11]:
def load_ds(fname='atis.train.pkl'):
    with open(fname, 'rb') as stream:
        ds, dicts = pickle.load(stream)
    print('Done  loading: ', fname)
    print('      samples: {:4d}'.format(len(ds['query'])))
    print('   vocab_size: {:4d}'.format(len(dicts['token_ids'])))
    print('   slot count: {:4d}'.format(len(dicts['slot_ids'])))
    print(' intent count: {:4d}'.format(len(dicts['intent_ids'])))
    return ds, dicts


train_ds, dicts = load_ds(os.path.join('.', 'atis.train.pkl'))
test_ds, dicts = load_ds(os.path.join('.', 'atis.test.pkl'))

Done  loading:  ./atis.train.pkl
      samples: 4978
   vocab_size:  943
   slot count:  129
 intent count:   26
Done  loading:  ./atis.test.pkl
      samples:  893
   vocab_size:  943
   slot count:  129
 intent count:   26


In [12]:
t2i, s2i, in2i = map(dicts.get, ['token_ids', 'slot_ids', 'intent_ids'])
i2t, i2s, i2in = map(lambda d: {d[k]: k for k in d.keys()}, [t2i, s2i, in2i])
query, slots, intent = map(train_ds.get, ['query', 'slot_labels', 'intent_labels'])

haa = []
for i in range(4978):
    for j in range(len(query[i])):
        haa.append('{:>33} {:>40}'.format(i2t[query[i][j]], i2s[slots[i][j]]))
print('*' * 74)
foo = []
for i in haa:
    token, tag = i.split()
    if token in ('EOS',' BOS'):
        foo.append('\n')
    else:
        foo.append('{} {}'.format(token, tag))

**************************************************************************


In [0]:
#!mkdir atis_data
with open('./atis_data/train.txt','w',encoding='utf8') as fp:
    for i in foo[:50000]:
        fp.writelines(i+'\n')

with open('./atis_data/test.txt','w',encoding='utf8') as fp:
    for i in foo[50000:60000]:
        fp.writelines(i+'\n')

with open('./atis_data/valid.txt','w',encoding='utf8') as fp:
    for i in foo[60000:]:
        fp.writelines(i+'\n')

### запуск обучения модели
Сама логика в https://gist.github.com/oserikov/d82ab401d278c454e9ff6e8779688f25
Это сырой код, скорее всего даже немного под копирайтом, поетому так.

In [14]:
!mkdir atis_out_test

mkdir: cannot create directory ‘atis_out_test’: File exists


In [43]:
!python3 /content/run_ner_atis.py --data_dir=atis_data/ \
 --bert_model=bert-large-uncased \
 --task_name=ner --output_dir=atis_out_07112016 --cache_dir atis_cache \
 --max_seq_length=32 \
 --do_train --num_train_epochs 10 --do_eval --warmup_proportion=0.2 \
 --do_lower_case --train_batch_size=32 # --fp16 # --loss_scale=0

11/07/2019 13:39:52 - INFO - __main__ -	 device: {}, n_gpu: {}, 16-bits training: cuda
11/07/2019 13:39:52 - INFO - pytorch_pretrained_bert.tokenization -	 loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
11/07/2019 13:39:52 - INFO - pytorch_pretrained_bert.modeling -	 loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased.tar.gz from cache at atis_cache/214d4777e8e3eb234563136cd3a49f6bc34131de836848454373fa43f10adc5e.abfbb80ee795a608acbf35c7bf2d2d58574df3887cdd94b355fc67e03fddba05
11/07/2019 13:39:52 - INFO - pytorch_pretrained_bert.modeling -	 extracting archive file atis_cache/214d4777e8e3eb234563136cd3a49f6bc34131de836848454373fa43f10adc5e.abfbb80ee795a608acbf35c7bf2d2d58574df3887cdd94b355fc67e03fddba05 to temp dir /tm